# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
citylist = pd.read_csv("Citylist.csv")
citylist


,Unnamed: 0,City Name,Lattitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Country,Date
0,0,Tuatapere,-46.13,167.68,39.00,83,14,3.00,NZ,1592522702
1,1,Te Anau,-45.42,167.72,36.82,80,16,1.16,NZ,1592522702
2,2,Pisco,-13.70,-76.22,64.22,82,75,11.41,PE,1592522450
3,3,Tarija,-21.54,-64.73,58.89,59,0,3.29,BO,1592522702
4,4,Tiksi,71.69,128.87,53.35,72,100,2.93,RU,1592522702
...,...,...,...,...,...,...,...,...,...,...
508,508,Sopela,43.38,-2.98,57.72,93,17,3.36,ES,1592522743
509,509,Krasnyy Chikoy,50.36,108.75,59.29,85,73,2.19,RU,1592522743
510,510,Acará,-1.96,-48.20,73.80,97,95,3.53,BR,1592522743
511,511,West Wendover,40.74,-114.07,71.60,26,1,11.41,US,1592522743


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)
locations = citylist[["Lattitude","Longitude"]]
humidity = citylist["Humidity"]
heatmap = gmaps.heatmap_layer(locations, weights=humidity)
fig = gmaps.figure()
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
#idealloc = citylist.loc[(citylist["Temperature"]>70)&（citylist["Temperature"]<80)&（citylist["Cloudiness"]==0)&（citylist["Windspeed"]<10)]
idealloc = citylist.loc[(citylist["Temperature"]>70)&(citylist["Temperature"]<80)&(citylist["Cloudiness"]==0)&(citylist["Windspeed"]<10)]
idealloc

,Unnamed: 0,City Name,Lattitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Country,Date
164,164,Tāybād,34.74,60.78,70.12,22,0,7.96,IR,1592522715
236,236,Cherskiy,68.75,161.30,76.10,29,0,8.77,RU,1592522721
323,323,Corrente,-3.74,-43.34,73.47,75,0,1.97,BR,1592522728
330,330,Jalu,29.03,21.55,78.55,26,0,7.56,LY,1592522729
495,495,Ubari,26.59,12.78,78.96,17,0,7.38,LY,1592522742


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [52]:
hotel_df = idealloc
hotel_df["Hotel Name"] = ""
hotel_df["Address"] = ""
hotel_df

C:\Users\schang\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\schang\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,City Name,Lattitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name,Address
164,164,Tāybād,34.74,60.78,70.12,22,0,7.96,IR,1592522715,,
236,236,Cherskiy,68.75,161.30,76.10,29,0,8.77,RU,1592522721,,
323,323,Corrente,-3.74,-43.34,73.47,75,0,1.97,BR,1592522728,,
330,330,Jalu,29.03,21.55,78.55,26,0,7.56,LY,1592522729,,
495,495,Ubari,26.59,12.78,78.96,17,0,7.38,LY,1592522742,,


In [53]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "68.75,161.30",
    "radius": 5000,
    #"rankby": "distance",
    "types": "lodging",
    "key": g_key
}

In [54]:
response = requests.get(base_url, params=params)
response.json()
print(response.json())

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 68.7568269, 'lng': 161.3276102}, 'viewport': {'northeast': {'lat': 68.7581758802915, 'lng': 161.3289591802915}, 'southwest': {'lat': 68.7554779197085, 'lng': 161.3262612197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '9cb89bf2401a45a90f7485ed308fa4d8e9c94858', 'name': 'Gostinitsa', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2340, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116763809125728590780">Константин Серповских</a>'], 'photo_reference': 'CmRaAAAArDWXwBL_HES_H9133xaM3OsdqvAgZmnksyNcw9cVRiosDeo72dhD0m5DtjU3Nv2aPvBkUoHQ9gt_iynBkRWNJHb5h5pPa21p0_OL1085WmfZ0hIjxeXnCZPyxm7FssfmEhA9jHsxExBeaSlRgApvi6TIGhRXy4L-Ee9fRU9m5Fwu5Kpj3Vs1vw', 'width': 4160}], 'place_id': 'ChIJXUtRCfEDJVoR97otZek0NzY', 'plus_code': {'compound_code': 'Q84H+P2 Chersky, Sakha Republic, Russia', 'global_code': '9VW3Q84H+P2'}, 'r

In [57]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get latitude from df
    lat = row['Lattitude']
    lng = row['Longitude']
    cityname = row['City Name']

    # add keyword to params dict
    params = {
    "location": str(lat) + "," + str(lng),
    "radius": 5000,
    #"rankby": "distance",
    "types": "lodging",
    "key": g_key
    }

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {cityname}.")
    response = requests.get(base_url, params=params).json()
  
    # extract results
    results = response['results']
    try:
        
        print(f"Closest {cityname} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Address'] = results[0]['vicinity']

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 164: Tāybād.
Closest Tāybād hotel is لی پوش jean center.
------------
Retrieving Results for Index 236: Cherskiy.


C:\Users\schang\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest Cherskiy hotel is Gostinitsa.
------------
Retrieving Results for Index 323: Corrente.
Closest Corrente hotel is Chapadinhense Hotel.
------------
Retrieving Results for Index 330: Jalu.
Closest Jalu hotel is Jalu Hotel.
------------
Retrieving Results for Index 495: Ubari.
Closest Ubari hotel is فندق أوباري.
------------


In [58]:
hotel_df

,Unnamed: 0,City Name,Lattitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name,Address
164,164,Tāybād,34.74,60.78,70.12,22,0,7.96,IR,1592522715,لی پوش jean center,تایباد،، بلوار امام خميني
236,236,Cherskiy,68.75,161.30,76.10,29,0,8.77,RU,1592522721,Gostinitsa,"ул.Пушкина, 34 кв.9, Chersky"
323,323,Corrente,-3.74,-43.34,73.47,75,0,1.97,BR,1592522728,Chapadinhense Hotel,"Avenida Oliveira Roma, 36-80, Chapadinha"
330,330,Jalu,29.03,21.55,78.55,26,0,7.56,LY,1592522729,Jalu Hotel,Jalu
495,495,Ubari,26.59,12.78,78.96,17,0,7.38,LY,1592522742,فندق أوباري,"Unnamed Road, أوباري،"


In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [68]:
# Add marker layer ontop of heat map
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lattitude', 'Longitude']].astype(float)
# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))